In [1]:
import pickle
from pathlib import Path

# Ruta al archivo results.pkl (ajústala según corresponda)
ruta = Path(r"E:\Aplicaciones\Nuevo_PRG\Compara_PRG\data\results\results_20250723_01.pkl")

# Cargar el archivo
with open(ruta, "rb") as f:
    data = pickle.load(f)

# Ver tipo de objeto cargado
print("Tipo:", type(data))

# Si es un diccionario, mostrar claves
if isinstance(data, dict):
    print("\nClaves principales:", list(data.keys())[:10])
    
    # Mostrar el tipo de los primeros valores
    for k, v in list(data.items())[:2]:
        print(f"\nClave: {k} → tipo: {type(v)}")
        if isinstance(v, dict):
            print("  Subclaves:", list(v.keys())[:10])

# Si es una lista, mostrar ejemplos
elif isinstance(data, list):
    print("\nEjemplo de elementos:", data[:5])

# En cualquier otro caso, mostrar contenido directo
else:
    print("\nContenido:", data)


Tipo: <class 'collections.defaultdict'>

Claves principales: ['PCP', 'PID1']

Clave: PCP → tipo: <class 'dict'>
  Subclaves: ['COTAS', 'CMG', 'GENTABLES', 'GENT', 'BESS', 'GENC']

Clave: PID1 → tipo: <class 'dict'>
  Subclaves: ['COTAS', 'CMG', 'GENTABLES', 'GENT', 'BESS', 'GENC']


In [2]:
cmg_df = data['PID1']['CMG']

# si está en Polars
import polars as pl
if isinstance(cmg_df, pl.DataFrame):
    cmg_df = cmg_df.to_pandas()

# filtrar nodos que empiezan con "bat_" (insensible a mayúsculas)
mask = cmg_df["Nombre_PLEXOS"].str.lower().str.startswith("bat_")
cmg_bat = cmg_df[mask]

print(cmg_bat.head())


                   Nombre_PLEXOS           5           6           7  \
106  BAT_LA_CABANA_EO_Mulchen220  102.609621  102.559523  104.858165   
107   BAT_MANZANO_FV_Polpaico220  102.987981  103.085304  105.939341   

              8           9          10          11          12          13  \
106  105.898861  106.164529  105.924199  105.151935  104.133441  105.476921   
107  106.295670  101.558571   99.158890   98.415784   97.881269   98.865823   

     ...         40         41         42         43         44         45  \
106  ...  61.659411  64.193347  68.593746  74.707181  74.338653  74.329639   
107  ...  59.768658  64.324980  70.274555  78.028562  78.312147  78.043243   

            46         47         48         49  
106  74.301513  74.663585  74.361388  65.250014  
107  78.120281  78.143269  78.254266  68.741756  

[2 rows x 46 columns]


In [ ]:
import os, sys, re, clr, csv
from shutil import copyfile
import pandas as pd
import subprocess as sp
from os.path import dirname, join
from System.IO import SearchOption as SearchOption
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, Animation, ArtistAnimation
from datetime import datetime
import numpy as np
from System import *

sys.path.append('C:/Program Files/Energy Exemplar/PLEXOS 10.0 API/')
clr.AddReference('PLEXOS_NET.Core')
clr.AddReference('EEUTILITY')
clr.AddReference('EnergyExemplar.PLEXOS.Utility')

from PLEXOS_NET.Core import *
from EEUTILITY.Enums import *
from EnergyExemplar.PLEXOS.Utility.Enums import *

copy_file = r"E:\Entrenamiento\PID_20250707\Modelos\DBSEN_PRGDIARIO_PID.xml"

if not os.path.exists(copy_file):
    print("❌ Archivo no encontrado")
    exit()

# Crear conexión al modelo
db = DatabaseCore()
db.DisplayAlerts = False
db.Connection(copy_file)

# Buscar IDs
collections = db.FetchAllCollectionIds()
attributes  = db.FetchAllAttributeEnums()
classes     = db.FetchAllClassIds()

In [ ]:
import pandas as pd

rows = []
for nod in db.GetChildMembers(collections['SystemNodes'], 'SEN'):
    # En tu entorno R08 esto funciona:
    lat = db.GetAttributeValue(classes['Node'], nod, attributes['Node.Latitude'], 0)[1]
    lon = db.GetAttributeValue(classes['Node'], nod, attributes['Node.Longitude'], 0)[1]
    rows.append({"Nodo": nod, "Lat": float(lat) if lat is not None else None,
                          "Lon": float(lon) if lon is not None else None})

df_nodes = pd.DataFrame(rows).drop_duplicates(subset=["Nodo"]).reset_index(drop=True)
print("df_nodes:", df_nodes.shape)
print(df_nodes.head())

# ─────────────────────────────────────────────────────────────
# 2) LÍNEAS con NodoFrom / NodoTo  → df_lines: [Linea, NodoFrom, NodoTo]
# ─────────────────────────────────────────────────────────────
mships_from = db.GetMemberships(collections['LineNodeFrom'])  # "(Linea) (NodoFrom)"
mships_to   = db.GetMemberships(collections['LineNodeTo'])    # "(Linea) (NodoTo)"

def parse_mships_line(mships, col_nodo):
    rows = []
    for mem in mships:
        op1 = mem.find('('); cp1 = mem.find(')')
        op2 = mem.find('(', op1+1); cp2 = mem.find(')', op2+1)
        linea = mem[op1+2:cp1-1]
        nodo  = mem[op2+2:cp2-1]
        rows.append({"Linea": linea, col_nodo: nodo})
    return pd.DataFrame(rows)

df_from = parse_mships_line(mships_from, "NodoFrom")
df_to   = parse_mships_line(mships_to,   "NodoTo")

df_lines = (
    pd.merge(df_from, df_to, on="Linea", how="outer")
      .dropna(subset=["NodoFrom","NodoTo"], how="any")        # opcional: solo líneas completas
      .drop_duplicates(subset=["Linea","NodoFrom","NodoTo"])
      .reset_index(drop=True)
)

print("df_lines:", df_lines.shape)
print(df_lines.head())

# ─────────────────────────────────────────────────────────────
# (Opcional) verificaciones rápidas para graficar
# ─────────────────────────────────────────────────────────────
# ¿Qué líneas tienen algún extremo sin coordenadas?
faltantes = (
    df_lines
    .merge(df_nodes.rename(columns={"Nodo":"NodoFrom","Lat":"LatFrom","Lon":"LonFrom"}), on="NodoFrom", how="left")
    .merge(df_nodes.rename(columns={"Nodo":"NodoTo","Lat":"LatTo","Lon":"LonTo"}), on="NodoTo", how="left")
    .query("LatFrom.isna() or LonFrom.isna() or LatTo.isna() or LonTo.isna()")
)
print("Líneas con extremos sin coordenadas:", len(faltantes))

import plotly.graph_objects as go

# === 1) Parámetros (Chile) ===
LAT_MIN, LAT_MAX = -56, -17
LON_MIN, LON_MAX = -76, -66

# === 2) Unir líneas con coordenadas de nodos ===
df_from_xy = df_lines.merge(
    df_nodes.rename(columns={"Nodo": "NodoFrom", "Lat": "LatFrom", "Lon": "LonFrom"}),
    on="NodoFrom", how="left"
)
df_lines_xy = df_from_xy.merge(
    df_nodes.rename(columns={"Nodo": "NodoTo", "Lat": "LatTo", "Lon": "LonTo"}),
    on="NodoTo", how="left"
).dropna(subset=["LatFrom","LonFrom","LatTo","LonTo"])

# (opcional) filtra a límites de Chile
df_lines_xy = df_lines_xy.query(
    "@LAT_MIN <= LatFrom <= @LAT_MAX and @LON_MIN <= LonFrom <= @LON_MAX and \
     @LAT_MIN <= LatTo   <= @LAT_MAX and @LON_MIN <= LonTo   <= @LON_MAX"
).reset_index(drop=True)

df_nodes_cl = df_nodes.dropna().query(
    "@LAT_MIN <= Lat <= @LAT_MAX and @LON_MIN <= Lon <= @LON_MAX"
).reset_index(drop=True)

# === 3) Figura ===
fig = go.Figure()

# Líneas (mismo color sobrio; sin nombres repetidos en leyenda)
for _, row in df_lines_xy.iterrows():
    fig.add_trace(go.Scattergeo(
        lon=[row["LonFrom"], row["LonTo"]],
        lat=[row["LatFrom"], row["LatTo"]],
        mode="lines",
        line=dict(width=2, color="#B0B6BE"),        # gris sutil
        hovertemplate=(
            "<b>%{customdata[0]}</b><br>"
            "From: %{customdata[1]}<br>"
            "To: %{customdata[2]}<extra></extra>"
        ),
        customdata=[[row["Linea"], row["NodoFrom"], row["NodoTo"]]],
        showlegend=False,                            # evita leyenda gigante
    ))

# Nodos (solo puntos, sin texto encima)
fig.add_trace(go.Scattergeo(
    lon=df_nodes_cl["Lon"],
    lat=df_nodes_cl["Lat"],
    text=df_nodes_cl["Nodo"],
    mode="markers",
    marker=dict(
        size=5,
        color="#1F6FEB",        # azul sobrio
        line=dict(width=0.5, color="white"),
        opacity=0.95
    ),
    hovertemplate="Nodo: %{text}<br>Lat: %{lat:.4f}<br>Lon: %{lon:.4f}<extra></extra>",
    showlegend=False,
))

# === 4) Estilo del mapa ===
fig.update_layout(
    title="<b>Red Eléctrica — Chile</b>",
    geo=dict(
        projection_type="mercator",
        showland=True, landcolor="#F8F9FA",
        showcountries=True, countrycolor="#A3AAB2",
        coastlinecolor="#A3AAB2",
        lakecolor="#E8F2FF", showlakes=True,
        lonaxis=dict(range=[LON_MIN, LON_MAX]),
        lataxis=dict(range=[LAT_MIN, LAT_MAX]),
        fitbounds="locations"
    ),
    margin=dict(l=20, r=20, t=60, b=20),
    height=1000, width=700,     # más grande y proporcionado
)

fig.show()


df_nodes: (218, 3)
          Nodo        Lat        Lon
0   AJahuel110 -33.714834 -70.691174
1   AJahuel154 -33.714834 -70.691174
2   AJahuel220 -33.714834 -70.691174
3   AJahuel500 -33.714834 -70.691174
4  Alfalfal220 -33.502260 -70.193360
df_lines: (313, 3)
                       Linea    NodoFrom         NodoTo
0  AJahuel110->Sauzal110_BP1  AJahuel110  Sauzal110_BP1
1       AJahuel154->Paine154  AJahuel154       Paine154
2  AJahuel154->Tuniche154_II  AJahuel154  Tuniche154_II
3     AJahuel220->AJahuel110  AJahuel220     AJahuel110
4     AJahuel220->AJahuel154  AJahuel220     AJahuel154
Líneas con extremos sin coordenadas: 0
